<a href="https://colab.research.google.com/github/cibercitizen1/cuda_hello/blob/main/Welcome_To_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [4]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-95juci6_
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-95juci6_
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4304 sha256=4748eee5c97dacede53cbc83c4c517361de21c22341f79b4c9cc31a348d6c52e
  Stored in directory: /tmp/pip-ephem-wheel-cache-0cbn1w2g/wheels/f3/08/cc/e2b5b0e1c92df07dbb50a6f024a68ce090f5e7b2316b41756d
Successfully built NVCCPlugin


In [5]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu
// -------------------------------------------------------------
// mainHello_1.cu
// -------------------------------------------------------------
#include <cstdio>
#include <iostream>

// -------------------------------------------------------------
// -------------------------------------------------------------
using namespace std;

// -------------------------------------------------------------
// Z -> () -> Z (for a kernel)
//
// [Z] -> () -> [Z] (for all the kernels)
// -------------------------------------------------------------
__global__ void test_kernel(int* p_input, int* p_output) {

  //
  // We wave 1-dim data (i.e. an array)
  // We have arranged one thread for one cell
  // both in the input array and in the output one
  // 
  // Therefore, we have to find out our thread index,
  // which equates to the cell number in the array
  // we have to manipulate
  //
  // The calculation is as follows:
  
  int idx = (blockIdx.x * blockDim.x) + threadIdx.x;

  // blockDim.x: the number of threads in the block for the x index
  // (which in this case is the only one)
  // times
  // blockIdx.x: number of block in for the x index
  // plus
  // threadIdx.x: the thread number within this block
  // Example: if we are the block number 3, each block has 16 threds
  // and the thread number is 7
  // The cell would be 3*16 + 7

  //
  // This is the calculation
  //
  p_output[idx] =  100 + p_input[idx];

}

// -------------------------------------------------------------
// -------------------------------------------------------------
int main() {

  //
  // input and output local arrays
  //
  const int N=1024;
  int numbers[N];
  int results[N];

	 int tam = N * sizeof(int);

  for (int i = 0; i <= N-1; i++) {
	numbers[i] = i;
	results[i] = -1;
  }

 
 

  //
  // get memory in the device
  //
  int* p_in;
  int* p_out;
  //
  cudaMalloc(&p_in, tam);
  cudaMalloc(&p_out, tam);

  //
  // timers, define and start to count
  //
  cudaEvent_t start; 
  cudaEvent_t end;
  cudaEventCreate(&start);
  cudaEventCreate(&end);
  
  cudaEventRecord(start);

  //
  // copy to device
  //
  cudaMemcpy(p_in, numbers, tam, cudaMemcpyHostToDevice);

  dim3 total_blocks( 4 );
  dim3 threads_per_block( N/4 );
  
  // dim3 total_blocks( 1 );
  // dim3 threads_per_block( N );


  //
  // start up the kernel(s)
  //
  test_kernel<<<total_blocks, threads_per_block>>>(p_in, p_out);

  //
  // wait for completion
  //
  cudaEventSynchronize(end);

  //
  // copy from device
  //
  cudaMemcpy(&results[0], p_out,  tam, cudaMemcpyDeviceToHost);
  
  //
  // record end moment, and calculate the elapsed time
  //
  cudaEventRecord(end);
  float time = 0;
  cudaEventElapsedTime(&time, start, end);

  //
  // results
  //

  cout << "results[1] : " << results[1] << endl;

  cout << "results[31] : " << results[31] << endl;
  cout << "results[32] : " << results[32] << endl;

  cout << "results[63] : " << results[63] << endl;
  cout << "results[64] : " << results[64] << endl;
  cout << "results[65] : " << results[65] << endl;
  cout << "results[" << N-1 << "] : " << results[N-1] << endl;


  cout << "start: " << start << endl;
  cout << "end: " << end << endl;
  //cout << (end - start) << endl;
  cout << "The time required : ";
  cout << time << endl;
} // main()
// -------------------------------------------------------------
// -------------------------------------------------------------
// -------------------------------------------------------------
// -------------------------------------------------------------

results[1] : -1
results[31] : -1
results[32] : -1
results[63] : -1
results[64] : -1
results[65] : -1
results[1023] : -1
start: 0
end: 0
The time required : 0



In [ ]:
!ls


sample_data  src


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%cu
/* This sample queries the properties of the CUDA devices present in the system via CUDA Runtime API. */

// includes, system
#include <stdlib.h>
#include <stdio.h>
#include <string.h>


#include <cuda.h>
#include <cuda_runtime_api.h>

// includes, project
// #include <cutil.h>

////////////////////////////////////////////////////////////////////////////////
// Program main
////////////////////////////////////////////////////////////////////////////////
int
main( int argc, char** argv) 
{
    printf("CUDA Device Query (Runtime API) version (CUDART static linking)\n");

    int deviceCount = 0;

	if (cudaGetDeviceCount(&deviceCount) != cudaSuccess) {
		printf("cudaGetDeviceCount failed! CUDA Driver and Runtime version may be mismatched.\n");
		printf("\nTest FAILED!\n");
		return 0;
	}

    // This function call returns 0 if there are no CUDA capable devices.
    if (deviceCount == 0)
        printf("There is no device supporting CUDA\n");

    int dev;
    for (dev = 0; dev < deviceCount; ++dev) {
        cudaDeviceProp deviceProp;
        cudaGetDeviceProperties(&deviceProp, dev);

        if (dev == 0) {
			// This function call returns 9999 for both major & minor fields, if no CUDA capable devices are present
            if (deviceProp.major == 9999 && deviceProp.minor == 9999)
                printf("There is no device supporting CUDA.\n");
            else if (deviceCount == 1)
                printf("There is 1 device supporting CUDA\n");
            else
                printf("There are %d devices supporting CUDA\n", deviceCount);
        }
        printf("\nDevice %d: \"%s\"\n", dev, deviceProp.name);
    #if CUDART_VERSION >= 2020
		int driverVersion = 0, runtimeVersion = 0;
		cudaDriverGetVersion(&driverVersion);
		printf("  CUDA Driver Version:                           %d.%d\n", driverVersion/1000, driverVersion%100);
		cudaRuntimeGetVersion(&runtimeVersion);
		printf("  CUDA Runtime Version:                          %d.%d\n", runtimeVersion/1000, runtimeVersion%100);
    #endif

        printf("  CUDA Capability Major revision number:         %d\n", deviceProp.major);
        printf("  CUDA Capability Minor revision number:         %d\n", deviceProp.minor);

		printf("  Total amount of global memory:                 %u bytes\n", deviceProp.totalGlobalMem);
    #if CUDART_VERSION >= 2000
        printf("  Number of multiprocessors:                     %d\n", deviceProp.multiProcessorCount);
        printf("  Number of cores:                               %d\n", 8 * deviceProp.multiProcessorCount);
    #endif
        printf("  Total amount of constant memory:               %u bytes\n", deviceProp.totalConstMem); 
        printf("  Total amount of shared memory per block:       %u bytes\n", deviceProp.sharedMemPerBlock);
        printf("  Total number of registers available per block: %d\n", deviceProp.regsPerBlock);
        printf("  Warp size:                                     %d\n", deviceProp.warpSize);
        printf("  Maximum number of threads per block:           %d\n", deviceProp.maxThreadsPerBlock);
        printf("  Maximum sizes of each dimension of a block:    %d x %d x %d\n",
               deviceProp.maxThreadsDim[0],
               deviceProp.maxThreadsDim[1],
               deviceProp.maxThreadsDim[2]);
        printf("  Maximum sizes of each dimension of a grid:     %d x %d x %d\n",
               deviceProp.maxGridSize[0],
               deviceProp.maxGridSize[1],
               deviceProp.maxGridSize[2]);
        printf("  Maximum memory pitch:                          %u bytes\n", deviceProp.memPitch);
        printf("  Texture alignment:                             %u bytes\n", deviceProp.textureAlignment);
        printf("  Clock rate:                                    %.2f GHz\n", deviceProp.clockRate * 1e-6f);
    #if CUDART_VERSION >= 2000
        printf("  Concurrent copy and execution:                 %s\n", deviceProp.deviceOverlap ? "Yes" : "No");
    #endif
    #if CUDART_VERSION >= 2020
        printf("  Run time limit on kernels:                     %s\n", deviceProp.kernelExecTimeoutEnabled ? "Yes" : "No");
        printf("  Integrated:                                    %s\n", deviceProp.integrated ? "Yes" : "No");
        printf("  Support host page-locked memory mapping:       %s\n", deviceProp.canMapHostMemory ? "Yes" : "No");
        printf("  Compute mode:                                  %s\n", deviceProp.computeMode == cudaComputeModeDefault ?
			                                                            "Default (multiple host threads can use this device simultaneously)" :
		                                                                deviceProp.computeMode == cudaComputeModeExclusive ?
																		"Exclusive (only one host thread at a time can use this device)" :
		                                                                deviceProp.computeMode == cudaComputeModeProhibited ?
																		"Prohibited (no host thread can use this device)" :
																		"Unknown");
    #endif
	}
    printf("\nTest PASSED\n");

    return 0;
}


CUDA Device Query (Runtime API) version (CUDART static linking)
There is 1 device supporting CUDA

Device 0: "Tesla T4"
  CUDA Driver Version:                           11.60
  CUDA Runtime Version:                          11.20
  CUDA Capability Major revision number:         7
  CUDA Capability Minor revision number:         5
  Total amount of global memory:                 2958950400 bytes
  Number of multiprocessors:                     40
  Number of cores:                               320
  Total amount of constant memory:               65536 bytes
  Total amount of shared memory per block:       49152 bytes
  Total number of registers available per block: 65536
  Warp size:                                     32
  Maximum number of threads per block:           1024
  Maximum sizes of each dimension of a block:    1024 x 1024 x 64
  Maximum sizes of each dimension of a grid:     2147483647 x 65535 x 65535
  Maximum memory pitch:                          2147483647 bytes
  Text

In [22]:
%%cu
// -------------------------------------------------------------
// mainHello_2.cu
// -------------------------------------------------------------
#include <cstdio>
#include <iostream>


// -------------------------------------------------------------
// -------------------------------------------------------------
using namespace std;

// -------------------------------------------------------------
// Shift to the left the contents of the array 
// -------------------------------------------------------------
__global__ void test_kernel( int* p_data, const int N ) {

  int aux;

  // find out my id
  int idx = (blockIdx.x * blockDim.x) + threadIdx.x;

  // let's slow down some threads
  if ( idx % 3 == 0) {
    // read "my own" element
    aux = p_data[idx]; 

    // just to keep the thread busy
    // and have the [idx] element with
    // a wrong value during this lapse
    for (int i=1; i<=10*10*10*10; i++ ) {
      p_data[idx] = -1234;
      p_data[idx] = floorf( sinf( i*i ) );
    } // for

    // finaly we put back the original value of [idx]
    p_data[idx] = aux; 
  } // if

  // supposedly, all threads stop here
  // so that when they pass this point
  // the array is as before
  __syncthreads(); 
  
  // caution: __syncthreads only syncs threads
  // corresponding to the same block
  // Thus, if there are several blocks one
  // thread per block migh read a wrong value
  // (i.e. not the original element but
  // the new one if already set by its neighbour)
  
  //
  // shift to the left in two steps: 1 read 2 write
  //
  // 1 read
  aux = p_data[ (idx+1) % N ];
  
  //
  // each thread should wait until the rest 
  // have read the value of its neighbour element.
  //
  __syncthreads(); 

  // 2 write
  p_data[idx] = aux;

} // ()

// -------------------------------------------------------------
// -------------------------------------------------------------
int main() {

  //
  // input and output local arrays
  //
  const int N=1024;
  int numbers[N];

	 int tam = N * sizeof(int);

  for (int i = 0; i <= N-1; i++) {
	  numbers[i] = i;
  }

  //
  // get memory in the device
  //
  int* p_data;
  //
  cudaMalloc(&p_data, tam);

  //
  // copy to device
  //
  cudaMemcpy(p_data, numbers, tam, cudaMemcpyHostToDevice);

  const int NUM_BLOCKS = 8;
  dim3 total_blocks( NUM_BLOCKS );
  dim3 threads_per_block( N / NUM_BLOCKS );
  
  //
  // timer
  //
  cudaEvent_t end;
  cudaEventCreate(&end);

  //
  // start up the kernel(s)
  //
  test_kernel<<<total_blocks, threads_per_block>>>(p_data, N);

  //
  // wait for completion
  //
  cudaEventSynchronize(end);

  //
  // copy from device
  //
  cudaMemcpy(&numbers[0], p_data,  tam, cudaMemcpyDeviceToHost);
  
  //
  // results
  //

  cout << "numbers[0] : " << numbers[0] << endl;
  cout << "numbers[1] : " << numbers[1] << endl;

  cout << "numbers[12] : " << numbers[12] << endl;
  cout << "numbers[13] : " << numbers[13] << endl;
  cout << "numbers[14] : " << numbers[14] << endl;

  cout << "numbers[31] : " << numbers[31] << endl;
  cout << "numbers[32] : " << numbers[32] << endl;
  cout << "numbers[33] : " << numbers[33] << endl;

  cout << "numbers[" << N-1 << "] : " << numbers[N-1] << endl;

  for (int i=0; i<=N-1; i++) {
     if (numbers[i] != (i+1) % N ) {
      cout << "element " << i << " is wrong" << endl;
      cout << "numbers[" << i << "] : " << numbers[i] << endl;
     }
  }

} // main()
// -------------------------------------------------------------
// -------------------------------------------------------------
// -------------------------------------------------------------
// -------------------------------------------------------------

numbers[0] : 1
numbers[1] : 2
numbers[12] : 13
numbers[13] : 14
numbers[14] : 15
numbers[31] : 32
numbers[32] : 33
numbers[33] : 34
numbers[1023] : 0
element 127 is wrong
numbers[127] : 129
element 255 is wrong
numbers[255] : 257
element 383 is wrong
numbers[383] : -1234
element 511 is wrong
numbers[511] : 513
element 639 is wrong
numbers[639] : 641
element 767 is wrong
numbers[767] : -1234
element 895 is wrong
numbers[895] : 897

